In [1]:
import tensorflow as tf
import numpy as np

/Users/akihiroaida/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [67]:
trX = np.linspace(-1, 1, 101, dtype=np.float64)
# オリジナルのコードではバイアス項を持たないが，ここではバイアス(=3)を含んだモデルとする
target_w = 2.
target_b = 3.
noise_gain = 0.33
trY = target_w * trX + target_b + np.random.randn(*trX.shape) * noise_gain

In [29]:
def model(X, w, b):
    # 線形予測子のモデル
    # オリジナルコードでは tf.muliply() を使っているが演算子 "*" でOK.
    # return tf.multiply(X, w) + b
    return X * w + b

In [30]:
# TensorFlow placeholders
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
# TensorFlow Variables
w = tf.Variable(0.0, name="weights")    # zero initialize
b = tf.Variable(0.0, name="bias")       # zero initialize

In [31]:
y_model = model(X, w, b)
cost = tf.reduce_mean(tf.square(Y - y_model))

In [32]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [33]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize variables (in this case just variable W)
    tf.global_variables_initializer().run()

    for i in range(100):
        for (x, y) in zip(trX, trY):
            sess.run(train_op, feed_dict={X: x, Y: y})

    final_w, final_b = sess.run([w, b])

In [34]:
print('w: {:.3}, b: {:.3}'.format(final_w, final_b))

w: 2.01, b: 3.05


In [ ]:
# estimatorを使ってみたり

In [69]:
# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
    # Build a linear model and predict values
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W * features['x'] + b
    # Loss sub-graph
    loss = tf.reduce_mean(tf.square(y - labels))
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
    # EstimatorSpec connects subgraphs we built to the
    # appropriate functionality.
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=y,
        loss=loss,
        train_op=train)

In [70]:
estimator = tf.estimator.Estimator(model_fn=model_fn, model_dir='tu2-model')

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'tu2-model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11426f710>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'tu2-model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11426f710>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [79]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": trX}, trY, batch_size=4, num_epochs=None, shuffle=True)

In [83]:
# train
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Restoring parameters from tu2-model/model.ckpt-2000


INFO:tensorflow:Restoring parameters from tu2-model/model.ckpt-2000


INFO:tensorflow:Saving checkpoints for 2001 into tu2-model/model.ckpt.


...

INFO:tensorflow:loss = 0.052781406929, step = 2901 (0.072 sec)


INFO:tensorflow:Saving checkpoints for 3000 into tu2-model/model.ckpt.


INFO:tensorflow:Saving checkpoints for 3000 into tu2-model/model.ckpt.


INFO:tensorflow:Loss for final step: 0.112374861555.


INFO:tensorflow:Loss for final step: 0.112374861555.


In [81]:
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn= tf.estimator.inputs.numpy_input_fn(
    {"x": trX}, trY, batch_size=4, num_epochs=1000, shuffle=False))
print("train metrics: %r"% train_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-29-08:20:46


INFO:tensorflow:Starting evaluation at 2017-12-29-08:20:46


INFO:tensorflow:Restoring parameters from tu2-model/model.ckpt-2000


INFO:tensorflow:Restoring parameters from tu2-model/model.ckpt-2000


INFO:tensorflow:Finished evaluation at 2017-12-29-08:20:55


INFO:tensorflow:Finished evaluation at 2017-12-29-08:20:55


INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.106778


INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.106778


train metrics: {'loss': 0.10677811, 'global_step': 2000}


In [87]:
print('w: {:.3}, b: {:.3}'.format(estimator.get_variable_value('W')[0], estimator.get_variable_value('b')[0]))

w: 2.03, b: 2.96


In [16]:
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split

In [90]:
# weights初期化のサポート関数
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

# オリジナルコードでは，バイアスをmodelに入れていないが，本コードは入れる．
def model(X, w, b):
    return tf.matmul(X, w) + b

In [91]:
# digits は，scikit-learn が用意する手書き文字データセット．pixel: 8 x 8
def load_data():
    digits = load_digits()
    digits_images = digits.data / 16.   # scaling to (0 .. 1)
    digits_target_ = []
    for i in range(len(digits.target)):
        target_one = np.zeros([10], dtype=np.float32)
        target_one[digits.target[i]] = 1.
        digits_target_.append(target_one)
    digits_target_onehot = np.asarray(digits_target_)
    return digits_images, digits_target_onehot

X, Y = load_data()

# train / test セットに分割する
trX, teX, trY, teY =  train_test_split(X, Y, test_size=0.2)

In [37]:
# TensorFlow placeholders
X_ph = tf.placeholder(tf.float32, [None, 64])
Y_ph = tf.placeholder(tf.float32, [None, 10])

# オリジナルコードでは，バイアスをmodelに入れていないが，本コードでは入れる．
w = init_weights([64, 10])
b = tf.zeros([10])

py_x = model(X_ph, w, b)

# コスト関数，オプティマイザ，予測
cost = tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y_ph)
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [40]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize variables (in this case just variable W)
    tf.global_variables_initializer().run()

    for i in range(100):
#         for (x, y) in zip(trX, trY):
#             sess.run(train_op, feed_dict={X_ph: x, Y_ph: y})
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            train_fd = {X_ph: trX[start:end], Y_ph: trY[start:end]}
            sess.run(train_op, feed_dict=train_fd)
        # numpy レベルで正解率(accuracy)を算定し出力
        if i % 10 == 0:
            print('step {:>3d}: accuracy = {:>8.3f}'.format(
                i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X_ph: teX}))))

step   0: accuracy =    0.872
step  10: accuracy =    0.958
step  20: accuracy =    0.969
step  30: accuracy =    0.972
step  40: accuracy =    0.972
step  50: accuracy =    0.972
step  60: accuracy =    0.972
step  70: accuracy =    0.972
step  80: accuracy =    0.972
step  90: accuracy =    0.972


In [154]:
# Declare list of features, we only have one real-valued feature
def model_fn2(features, labels, mode):
    print(features['x'].shape)
    print(labels.shape)
    
    # オリジナルコードでは，バイアスをmodelに入れていないが，本コードでは入れる．
    w = tf.get_variable('W', [64, 10], dtype=tf.float64)
    b = tf.get_variable("b", [10], dtype=tf.float64, initializer=tf.zeros_initializer)

    py_x = tf.matmul(features['x'], w) + b

    # コスト関数，オプティマイザ，予測
#     cost = tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=labels)
    loss =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=labels))
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    # Training sub-graph
    global_step = tf.train.get_global_step()
    train = tf.group(train_op,
                   tf.assign_add(global_step, 1))
    # EstimatorSpec connects subgraphs we built to the
    # appropriate functionality.
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=py_x,
        loss=loss,
        train_op=train)

In [155]:
estimator = tf.estimator.Estimator(model_fn=model_fn2, model_dir='tu2-model2')

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'tu2-model2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x115e694a8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'tu2-model2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x115e694a8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [156]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": trX}, trY, batch_size=128, num_epochs=None, shuffle=True)

In [157]:
# train
estimator.train(input_fn=input_fn, steps=1000)

(128, 64)
(128, 10)
INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into tu2-model2/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1 into tu2-model2/model.ckpt.


INFO:tensorflow:loss = 2.52878923892, step = 1


...

INFO:tensorflow:loss = 1.24795470722, step = 901 (0.117 sec)


INFO:tensorflow:Saving checkpoints for 1000 into tu2-model2/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1000 into tu2-model2/model.ckpt.


INFO:tensorflow:Loss for final step: 1.19704985792.


INFO:tensorflow:Loss for final step: 1.19704985792.


In [158]:
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn= tf.estimator.inputs.numpy_input_fn(
    {"x": teX}, teY, batch_size=4, num_epochs=1000, shuffle=False))
print("train metrics: %r"% train_metrics)

(?, 64)
(?, 10)
INFO:tensorflow:Starting evaluation at 2017-12-29-08:58:26


INFO:tensorflow:Starting evaluation at 2017-12-29-08:58:26


INFO:tensorflow:Restoring parameters from tu2-model2/model.ckpt-1000


INFO:tensorflow:Restoring parameters from tu2-model2/model.ckpt-1000


INFO:tensorflow:Finished evaluation at 2017-12-29-08:59:02


INFO:tensorflow:Finished evaluation at 2017-12-29-08:59:02


INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.21103


INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.21103


train metrics: {'loss': 1.2110331, 'global_step': 1000}


In [159]:
estimator.get_variable_names()

['W', 'b', 'global_step']

In [161]:
preW = estimator.get_variable_value('W')
preb = estimator.get_variable_value('b')

In [163]:
preW.shape, preb.shape

((64, 10), (10,))